In [1]:
import sys
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import operator
from matplotlib import pylab as plt
from datetime import datetime
import time
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('data/train_set.csv')  # 读取训练数据
data.head()

,user_id,sku_id,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,...,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,label
0,202633.0,12564.0,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,...,20.866667,5167.6,1.0,0.0260,0.0,0.0,0.0,0.0,1.0,1.0
1,218498.0,149854.0,8.0,4.0,0.0,0.0,0.0,0.0,4.0,2.0,...,20.866667,5167.6,1.0,0.0403,0.0,0.0,0.0,0.0,1.0,1.0
2,221842.0,75877.0,8.0,3.0,0.0,0.0,0.0,0.0,5.0,79.0,...,20.866667,5167.6,1.0,0.0245,0.0,0.0,0.0,0.0,1.0,1.0
3,222886.0,154636.0,8.0,20.0,1.0,0.0,0.0,0.0,26.0,10.0,...,20.866667,5167.6,1.0,0.0208,0.0,0.0,0.0,0.0,1.0,1.0
4,235240.0,38222.0,8.0,30.0,1.0,0.0,0.0,0.0,28.0,55.0,...,20.866667,5167.6,1.0,0.0166,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
data.columns

Index(['user_id', 'sku_id', 'cate', 'action_before_3_1.0_x',
       'action_before_3_2.0_x', 'action_before_3_3.0_x',
       'action_before_3_4.0_x', 'action_before_3_5.0_x',
       'action_before_3_6.0_x', 'action_before_3_1.0_y',
       ...
       'cate_action_5_mean', 'cate_action_6_mean', 'has_bad_comment',
       'bad_comment_rate', 'comment_num_0', 'comment_num_1', 'comment_num_2',
       'comment_num_3', 'comment_num_4', 'label'],
      dtype='object', length=237)

In [13]:
data_x = data.loc[:,data.columns != 'label']  # 将训练数据集分成训练模型和测试集
data_y = data.loc[:,data.columns == 'label']
data_x.head()

,user_id,sku_id,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,...,cate_action_4_mean,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4
0,202633.0,12564.0,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,...,8.4,20.866667,5167.6,1.0,0.0260,0.0,0.0,0.0,0.0,1.0
1,218498.0,149854.0,8.0,4.0,0.0,0.0,0.0,0.0,4.0,2.0,...,8.4,20.866667,5167.6,1.0,0.0403,0.0,0.0,0.0,0.0,1.0
2,221842.0,75877.0,8.0,3.0,0.0,0.0,0.0,0.0,5.0,79.0,...,8.4,20.866667,5167.6,1.0,0.0245,0.0,0.0,0.0,0.0,1.0
3,222886.0,154636.0,8.0,20.0,1.0,0.0,0.0,0.0,26.0,10.0,...,8.4,20.866667,5167.6,1.0,0.0208,0.0,0.0,0.0,0.0,1.0
4,235240.0,38222.0,8.0,30.0,1.0,0.0,0.0,0.0,28.0,55.0,...,8.4,20.866667,5167.6,1.0,0.0166,0.0,0.0,0.0,0.0,1.0


In [14]:
data_y.head()

,label
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [15]:
x_train, x_test, y_train, y_test = train_test_split(data_x,data_y,test_size = 0.2, random_state = 0)  # 数据切分成两份，训练和测试，8：2切分
print(x_train.shape)
print(x_test.shape)

(11695, 236)
(2924, 236)


In [16]:
# 上面测试集刚好有近3000条，二切分成训练时的验证和预测
x_val = x_test.iloc[:1500,:]
y_val = y_test.iloc[:1500,:]

x_test = x_test.iloc[1500:,:] 
y_test = y_test.iloc[1500:,:]

In [17]:
print(x_val.shape)
print(x_test.shape)

(1500, 236)
(1424, 236)


In [18]:
del x_train['user_id']
del x_train['sku_id']

del x_val['user_id']
del x_val['sku_id']

x_train.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_4_mean,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4
2157,4.0,2.0,0.0,0.0,0.0,0.0,4.0,48.0,0.0,0.0,...,25.300000,52.133333,16112.033333,1.0,0.0344,0.0,0.0,0.0,0.0,1.0
2464,10.0,2.0,0.0,0.0,0.0,0.0,7.0,288.0,0.0,0.0,...,0.433333,5.066667,1273.500000,1.0,0.0132,0.0,0.0,0.0,0.0,1.0
10326,8.0,2.0,1.0,1.0,0.0,0.0,3.0,42.0,2.0,1.0,...,35.233333,149.266667,32299.233333,1.0,0.0213,0.0,0.0,0.0,0.0,1.0
7025,8.0,2.0,0.0,0.0,0.0,0.0,2.0,36.0,1.0,0.0,...,25.733333,116.800000,24942.666667,0.0,0.0000,0.0,0.0,1.0,0.0,0.0
6625,7.0,13.0,0.0,0.0,0.0,0.0,22.0,92.0,5.0,1.0,...,17.000000,36.166667,9447.266667,1.0,0.0800,0.0,0.0,0.0,0.0,1.0


In [19]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_val, label=y_val)

In [20]:
param = {'n_estimators': 4000, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0, 'subsample': 1.0, 
             'colsample_bytree': 0.8, 'scale_pos_weight':10, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic',
             'eval_metric':'auc'}

In [22]:
num_round = param['n_estimators']

plst = param.items()
evallist = [(dtrain, 'train'), (dvalid, 'eval')]
best = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=10)  # 寻找最优参
best.save_model('bst.model')

[0]	train-auc:0.938547	eval-auc:0.934522
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 10 rounds.
[1]	train-auc:0.947568	eval-auc:0.944769
[2]	train-auc:0.952758	eval-auc:0.949358
[3]	train-auc:0.955704	eval-auc:0.952481
[4]	train-auc:0.95525	eval-auc:0.952343
[5]	train-auc:0.957462	eval-auc:0.95475
[6]	train-auc:0.957636	eval-auc:0.955133
[7]	train-auc:0.958327	eval-auc:0.955077
[8]	train-auc:0.958339	eval-auc:0.95549
[9]	train-auc:0.958235	eval-auc:0.955479
[10]	train-auc:0.958922	eval-auc:0.955668
[11]	train-auc:0.959378	eval-auc:0.956423
[12]	train-auc:0.959666	eval-auc:0.956724
[13]	train-auc:0.959674	eval-auc:0.957192
[14]	train-auc:0.960227	eval-auc:0.957447
[15]	train-auc:0.960316	eval-auc:0.957583
[16]	train-auc:0.961338	eval-auc:0.959102
[17]	train-auc:0.961986	eval-auc:0.958905
[18]	train-auc:0.962297	eval-auc:0.959365
[19]	train-auc:0.962798	eval-auc:0.959863
[20]	train-auc:0.963266	eval-auc

In [23]:
best.attributes()

{'best_iteration': '135',
 'best_msg': '[135]\ttrain-auc:0.983497\teval-auc:0.975266',
 'best_score': '0.975266'}

In [24]:
def create_feature_map(features):
    outfile = open(r'xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()


features = list(x_train.columns[:])
create_feature_map(features)

In [27]:
def feature_importance(best_xgb):
    importance = best_xgb.get_fscore(fmap=r'xgb.fmap')
    importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)

    df = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df['fscore'] = df['fscore'] / df['fscore'].sum()
    file_name = 'data/feature_importance_' + str(datetime.now().date())[5:] + '.csv'
    df.to_csv(file_name)

In [29]:
feature_importance(best)

In [30]:
fi = pd.read_csv('data/feature_importance_02-05.csv')
fi.sort_values("fscore", inplace=True, ascending=False)
fi.head()

,Unnamed: 0,feature,fscore
0,0,brand,0.077852
1,1,action_before_10_5.0_x,0.041611
2,2,bad_comment_rate,0.038926
3,3,product_action_5_ratio,0.028188
4,4,user_lv_cd_2,0.025503


In [31]:
x_test.head()

,user_id,sku_id,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,...,cate_action_4_mean,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4
6765,235394.0,24371.0,8.0,8.0,0.0,0.0,0.0,1.0,18.0,12.0,...,25.733333,116.8,24942.666667,1.0,0.0206,0.0,0.0,0.0,0.0,1.0
13767,272948.0,108907.0,4.0,8.0,0.0,0.0,0.0,0.0,7.0,0.0,...,61.633333,143.9,41669.233333,0.0,0.0000,0.0,0.0,1.0,0.0,0.0
9672,245846.0,63026.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,47.0,...,1.100000,9.0,2361.600000,1.0,0.0938,0.0,0.0,0.0,1.0,0.0
9116,272178.0,66704.0,9.0,2.0,0.0,0.0,0.0,0.0,4.0,112.0,...,10.400000,32.5,7264.200000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0
10055,216485.0,131364.0,6.0,2.0,1.0,1.0,0.0,0.0,6.0,28.0,...,38.233333,88.0,17558.033333,1.0,0.0473,0.0,0.0,0.0,0.0,1.0


In [32]:
users = x_test[['user_id', 'sku_id', 'cate']].copy()
del x_test['user_id']
del x_test['sku_id']
x_test_DMatrix = xgb.DMatrix(x_test)
y_pred = bst.predict(x_test_DMatrix, ntree_limit=bst.best_ntree_limit)

In [33]:
x_test['pred_label'] = y_pred
x_test.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label
6765,8.0,8.0,0.0,0.0,0.0,1.0,18.0,12.0,0.0,0.0,...,116.8,24942.666667,1.0,0.0206,0.0,0.0,0.0,0.0,1.0,0.453736
13767,4.0,8.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,...,143.9,41669.233333,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.002793
9672,10.0,1.0,0.0,0.0,0.0,0.0,0.0,47.0,0.0,0.0,...,9.0,2361.600000,1.0,0.0938,0.0,0.0,0.0,1.0,0.0,0.000167
9116,9.0,2.0,0.0,0.0,0.0,0.0,4.0,112.0,0.0,0.0,...,32.5,7264.200000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000225
10055,6.0,2.0,1.0,1.0,0.0,0.0,6.0,28.0,1.0,0.0,...,88.0,17558.033333,1.0,0.0473,0.0,0.0,0.0,0.0,1.0,0.000507


In [34]:
def label(column):
    if column['pred_label'] > 0.5:
        #rint ('yes')
        column['pred_label'] = 1
    else:
        column['pred_label'] = 0
    return column
x_test = x_test.apply(label,axis = 1)
x_test.head()  

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_5_mean,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label
6765,8.0,8.0,0.0,0.0,0.0,1.0,18.0,12.0,0.0,0.0,...,116.8,24942.666667,1.0,0.0206,0.0,0.0,0.0,0.0,1.0,0.0
13767,4.0,8.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,...,143.9,41669.233333,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.0
9672,10.0,1.0,0.0,0.0,0.0,0.0,0.0,47.0,0.0,0.0,...,9.0,2361.600000,1.0,0.0938,0.0,0.0,0.0,1.0,0.0,0.0
9116,9.0,2.0,0.0,0.0,0.0,0.0,4.0,112.0,0.0,0.0,...,32.5,7264.200000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
10055,6.0,2.0,1.0,1.0,0.0,0.0,6.0,28.0,1.0,0.0,...,88.0,17558.033333,1.0,0.0473,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
x_test['true_label'] = y_test
x_test.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,cate_action_6_mean,has_bad_comment,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label,true_label
6765,8.0,8.0,0.0,0.0,0.0,1.0,18.0,12.0,0.0,0.0,...,24942.666667,1.0,0.0206,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13767,4.0,8.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,...,41669.233333,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9672,10.0,1.0,0.0,0.0,0.0,0.0,0.0,47.0,0.0,0.0,...,2361.600000,1.0,0.0938,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9116,9.0,2.0,0.0,0.0,0.0,0.0,4.0,112.0,0.0,0.0,...,7264.200000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10055,6.0,2.0,1.0,1.0,0.0,0.0,6.0,28.0,1.0,0.0,...,17558.033333,1.0,0.0473,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [36]:
x_test['user_id'] = users['user_id']
x_test['sku_id'] = users['sku_id']
x_test.head()

,cate,action_before_3_1.0_x,action_before_3_2.0_x,action_before_3_3.0_x,action_before_3_4.0_x,action_before_3_5.0_x,action_before_3_6.0_x,action_before_3_1.0_y,action_before_3_2.0_y,action_before_3_3.0_y,...,bad_comment_rate,comment_num_0,comment_num_1,comment_num_2,comment_num_3,comment_num_4,pred_label,true_label,user_id,sku_id
6765,8.0,8.0,0.0,0.0,0.0,1.0,18.0,12.0,0.0,0.0,...,0.0206,0.0,0.0,0.0,0.0,1.0,0.0,0.0,235394.0,24371.0
13767,4.0,8.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,...,0.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,272948.0,108907.0
9672,10.0,1.0,0.0,0.0,0.0,0.0,0.0,47.0,0.0,0.0,...,0.0938,0.0,0.0,0.0,1.0,0.0,0.0,0.0,245846.0,63026.0
9116,9.0,2.0,0.0,0.0,0.0,0.0,4.0,112.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,272178.0,66704.0
10055,6.0,2.0,1.0,1.0,0.0,0.0,6.0,28.0,1.0,0.0,...,0.0473,0.0,0.0,0.0,0.0,1.0,0.0,0.0,216485.0,131364.0


In [37]:
# 所有购买用户
all_user_set = x_test[x_test['true_label']==1]['user_id'].unique()
print (len(all_user_set))
# 所有预测购买的用户
all_user_test_set = x_test[x_test['pred_label'] == 1]['user_id'].unique()
print (len(all_user_test_set))
all_user_test_item_pair = x_test[x_test['pred_label'] == 1]['user_id'].map(str) + '-' + x_test[x_test['pred_label'] == 1]['sku_id'].map(str)
all_user_test_item_pair = np.array(all_user_test_item_pair)
print (len(all_user_test_item_pair))

126
248
267


In [38]:
pos, neg = 0,0
for user_id in all_user_test_set:
    if user_id in all_user_set:
        pos += 1
    else:
        neg += 1
all_user_acc = 1.0 * pos / (pos + neg)
all_user_recall = 1.0 * pos / len(all_user_set)
print ('所有用户中预测购买用户的准确率为 ' + str(all_user_acc))
print ('所有用户中预测购买用户的召回率' + str(all_user_recall))

所有用户中预测购买用户的准确率为 0.4838709677419355
所有用户中预测购买用户的召回率0.9523809523809523


In [40]:
#所有实际商品对
all_user_item_pair = x_test[x_test['true_label']==1]['user_id'].map(str) + '-' + x_test[x_test['true_label']==1]['sku_id'].map(str)
all_user_item_pair = np.array(all_user_item_pair)
print (len(all_user_item_pair))
pos, neg = 0, 0
for user_item_pair in all_user_test_item_pair:
    #print (user_item_pair)
    if user_item_pair in all_user_item_pair:
        pos += 1
    else:
        neg += 1
all_item_acc = 1.0 * pos / ( pos + neg)
all_item_recall = 1.0 * pos / len(all_user_item_pair)
print ('所有用户中预测购买商品的准确率为 ' + str(all_item_acc))
print ('所有用户中预测购买商品的召回率' + str(all_item_recall))
F11 = 6.0 * all_user_recall * all_user_acc / (5.0 * all_user_recall + all_user_acc)
F12 = 5.0 * all_item_acc * all_item_recall / (2.0 * all_item_recall + 3 * all_item_acc)
score = 0.4 * F11 + 0.6 * F12
print ('F11=' + str(F11))
print ('F12=' + str(F12))
print ('score=' + str(score))

144
所有用户中预测购买商品的准确率为 0.5131086142322098
所有用户中预测购买商品的召回率0.9513888888888888
F11=0.527086383601757
F12=0.7091097308488614
score=0.6363003919500196
